Show the first name and the email address of customer with CompanyName 'Bike World'

```mysql
SELECT FirstName, EmailAddress
FROM Customer
WHERE CompanyName = 'Bike World'
```

Show the CompanyName for all customers with an address in City 'Dallas'.

```mysql
SELECT DISTINCT CompanyName
FROM Customer JOIN CustomerAddress cusAdd 
    ON Customer.CustomerID = cusAdd .CustomerID
JOIN Address ON cusAdd.AddressID = Address.AddressID
WHERE City = 'Dallas'
```

How many items with ListPrice more than $1000 have been sold?

```mysql
SELECT COUNT(*)
FROM Product
WHERE ListPrice > 1000
```

Give the CompanyName of those customers with orders over $100000. Include the subtotal plus tax plus freight.

```mysql
SELECT DISTINCT CompanyName, SubTotal, TaxAmt, Freight
FROM Product JOIN SalesOrderDetail detail ON Product.ProductID = detail.ProductID
JOIN SalesOrderHeader header ON detail.SalesOrderID = header.SalesOrderID
JOIN Customer ON Customer.CustomerID = header.CustomerID
WHERE (SubTotal + TaxAmt + Freight) > 100000
```

Find the number of left racing socks ('Racing Socks, L') ordered by CompanyName 'Riding Cycles'

```mysql
SELECT SUM(OrderQty)
FROM Product JOIN SalesOrderDetail detail ON Product.ProductID = detail.ProductID
JOIN SalesOrderHeader header ON detail.SalesOrderID = header.SalesOrderID
JOIN Customer ON Customer.CustomerID = header.CustomerID
WHERE Name = 'Racing Socks, L'
    AND CompanyName = 'Riding Cycles'
```

A "Single Item Order" is a customer order where only one item is ordered. Show the SalesOrderID and the UnitPrice for every Single Item Order.

```mysql
SELECT SalesOrderID, UnitPrice
FROM SalesOrderDetail x
WHERE SalesOrderID in (SELECT SalesOrderID
                       FROM SalesOrderDetail y
                       GROUP BY SalesOrderID
                       HAVING COUNT(*) = 1)
```

Where did the racing socks go? List the product name and the CompanyName for all Customers who ordered ProductModel 'Racing Socks'.

```mysql
SELECT Product.Name, CompanyName
FROM Product JOIN SalesOrderDetail detail ON Product.ProductID = detail.ProductID
JOIN ProductModel prodMod ON Product.ProductModelID = prodMod.ProductModelID
JOIN SalesOrderHeader header ON detail.SalesOrderID = header.SalesOrderID
JOIN Customer ON Customer.CustomerID = header.CustomerID
WHERE prodMod.Name = 'Racing Socks'
```

Show the product description for culture 'fr' for product with ProductID 736.

```mysql
SELECT Description
FROM ProductDescription description JOIN ProductModelProductDescription PMPD 
    ON description.ProductDescriptionID = PMPD.ProductDescriptionID
JOIN ProductModel prodMod ON PMPD.ProductModelID = prodMod.ProductModelID
JOIN Product ON Product.ProductModelID = prodMod.ProductModelID
WHERE ProductID = 736 
    AND Culture = 'fr'
```

Use the SubTotal value in SaleOrderHeader to list orders from the largest to the smallest. For each order show the CompanyName and the SubTotal and the total weight of the order.

```mysql
SELECT CompanyName, SubTotal, TotalWeight
FROM (SELECT SalesOrderID, SUM(Weight * OrderQty) as TotalWeight
      FROM Product JOIN SalesOrderDetail detail 
          ON Product.ProductID = detail.ProductID
      GROUP BY SalesOrderID) x
JOIN SalesOrderHeader header ON x.SalesOrderID = header.SalesOrderID 
JOIN Customer ON header.CustomerID = Customer.CustomerID
ORDER BY SubTotal DESC
```

How many products in ProductCategory 'Cranksets' have been sold to an address in 'London'?

```mysql
SELECT SUM(OrderQty)
FROM ProductCategory categ JOIN Product 
    ON categ.ProductCategoryID = Product.ProductCategoryID
JOIN SalesOrderDetail detail ON Product.ProductID = detail.ProductID
JOIN ProductModel prodMod ON Product.ProductModelID = prodMod.ProductModelID
JOIN SalesOrderHeader header ON detail.SalesOrderID = header.SalesOrderID
JOIN Address ON Address.AddressID = header.ShipToAddressID
WHERE City = 'London' AND categ.Name = 'Cranksets'
```

For every customer with a 'Main Office' in Dallas show AddressLine1 of the 'Main Office' and AddressLine1 of the 'Shipping' address - if there is no shipping address leave it blank. Use one row per customer.

```mysql
SELECT main.AddressLine1, shipping.AddressLine1
FROM (SELECT CustomerID, AddressLine1, City
      FROM CustomerAddress JOIN Address 
          ON CustomerAddress.AddressID = Address.AddressID
      WHERE AddressType = 'Main Office') main
LEFT JOIN (SELECT CustomerID, AddressLine1
           FROM CustomerAddress JOIN Address 
               ON CustomerAddress.AddressID = Address.AddressID
           WHERE AddressType = 'Shipping') shipping 
               ON main.CustomerID = shipping.CustomerID
WHERE main.city = 'Dallas'
```

For each order show the SalesOrderID and SubTotal calculated three ways:
1. From the SalesOrderHeader
2. Sum of OrderQty*UnitPrice
3. Sum of OrderQty*ListPrice

```mysql
SELECT header.SalesOrderID, SubTotal, SubTotalUnit, SubTotalList
FROM SalesOrderHeader header
JOIN (SELECT 
          SalesOrderID, 
          SUM(OrderQty * UnitPrice) AS SubTotalUnit, 
          SUM(OrderQty * ListPrice) AS SubTotalList
      FROM SalesOrderDetail detail 
      JOIN Product ON detail.ProductID = Product.ProductID
      GROUP BY SalesOrderID) grouped
ON header.SalesOrderID = grouped.SalesOrderID
```

Show the best selling item by value.

```mysql
SELECT Name, SUM(OrderQty * UnitPrice) as value
FROM Product JOIN SalesOrderDetail detail ON Product.ProductID = detail.ProductID
GROUP BY Name
ORDER BY value DESC
```

Show how many orders are in the following ranges (in $):

```mysql
SELECT Range, SUM(OrderQty) as [Num Orders], SUM(SubTotal) AS [Total Value]
FROM (SELECT 
          CASE WHEN SubTotal BETWEEN 0 AND 99 THEN '0-99'
              WHEN SubTotal BETWEEN 100 AND 999 THEN '100-999'
              WHEN SubTotal BETWEEN 1000 AND 9999 THEN '1000-9999'
              ELSE '10000-' END AS Range, 
          OrderQty, 
          SubTotal
      FROM SalesOrderHeader header JOIN SalesOrderDetail detail 
          ON header.SalesOrderID = detail.SalesOrderID) x
GROUP BY Range
```

Identify the three most important cities. Show the break down of top level product category against city.

```mysql
SELECT City, category.Name, SUM(OrderQty) AS Quantity
FROM ProductCategory category JOIN Product 
    ON category.ProductCategoryID = product.ProductCategoryID
JOIN SalesOrderDetail detail ON Product.ProductID = detail.ProductID
JOIN ProductModel prodMod ON Product.ProductModelID = prodMod.ProductModelID
JOIN SalesOrderHeader header ON detail.SalesOrderID = header.SalesOrderID
JOIN Address  ON Address.AddressID = header.ShipToAddressID
WHERE City IN (SELECT City
               FROM (SELECT City, RANK() OVER (ORDER BY Subtotal DESC) AS rank 
                     FROM (SELECT City, SUM(Subtotal) AS Subtotal
                           FROM Address JOIN SalesOrderHeader header 
                               ON Address.AddressID = header.ShipToAddressID
                           GROUP BY City) x) y
               WHERE rank <= 3)
GROUP BY City, category.Name
ORDER BY City, Quantity DESC
```

List the SalesOrderNumber for the customers 'Good Toys' and 'Bike World'

```mysql
SELECT SalesOrderID
FROM SalesOrderHeader header JOIN Customer 
    ON header.CustomerID = Customer.CustomerID
WHERE CompanyName IN ('Good Toys', 'Bike World')
```

List the ProductName and the quantity of what was ordered by 'Futuristic Bikes'

```mysql
SELECT Product.Name
FROM Product JOIN SalesOrderDetail detail ON Product.ProductID = detail.ProductID
JOIN ProductModel prodMod ON Product.ProductModelID = prodMod.ProductModelID
JOIN SalesOrderHeader header ON detail.SalesOrderID = header.SalesOrderID
JOIN Customer ON header.CustomerID = Customer.CustomerID
WHERE CompanyName = 'Futuristic Bikes'
```

List the name and addresses of companies containing the word 'Bike' (upper or lower case) and companies containing 'cycle' (upper or lower case). Ensure that the 'bike's are listed before the 'cycles's.

```mysql
SELECT CompanyName, AddressLine1, AddressLine2, City
FROM Customer JOIN SalesOrderHeader header
    ON header.CustomerID = Customer.CustomerID
JOIN Address ON header.ShipToAddressID = Address.AddressID
WHERE LOWER(CompanyName) LIKE '%bike%' OR LOWER(CompanyName) LIKE '%cycle%'
ORDER BY 
    CASE WHEN LOWER(CompanyName) LIKE '%bike%' THEN 1
    ELSE 2 END
```

Show the total order value for each CountyRegion. List by value with the highest first.

```mysql
SELECT CountyRegion, SUM(SubTotal) as Value
FROM Address JOIN SalesOrderHeader header 
    ON Address.AddressID = header.ShipToAddressID
GROUP BY CountyRegion
ORDER BY Value DESC
```

Find the best customer in each region.

```mysql
SELECT CountyRegion, CompanyName
FROM Address JOIN SalesOrderHeader header 
    ON Address.AddressID = header.ShipToAddressID
JOIN Customer ON header.CustomerID = Customer.CustomerID
WHERE SubTotal >= ALL(SELECT SubTotal
                      FROM SalesOrderHeader otherHeader JOIN Address otherAddress
                      ON otherAddress.AddressID = otherHeader.ShipToAddressID
                      WHERE Address.CountyRegion = otherAddress.CountyRegion)
ORDER BY CountyRegion
```

```mysql
SELECT CountyRegion, CompanyName
FROM (SELECT 
          CountyRegion, 
          CompanyName, 
          RANK() OVER (PARTITION BY CountyRegion ORDER BY SubTotal DESC) AS rank
      FROM Address JOIN SalesOrderHeader header 
          ON Address.AddressID = header.ShipToAddressID
      JOIN Customer ON header.CustomerID = Customer.CustomerID) x
WHERE rank = 1
```